# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd 

url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [ ]:
df_filtered = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]
df_filtered.head()

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [ ]:
df_response = df[df['Response'] == 'Yes']
avg_claim_amount = df.groupby(['Policy','Gender'])['Total Claim Amount'].mean().round(2).reset_index()
avg_claim_amount

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [ ]:
state_count = df.groupby('State')['Customer'].count().reset_index()
state_count_over_500 = state_count[state_count['Customer'] > 500].sort_values(by='Customer', ascending=False)
print(state_count_over_500)

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
clv_stats = df.groupby(['Gender','Education']) ['Customer Lifetime Value'].agg(['max','min','median']).round(2).reset_index()
clv_stats = clv_stats.rename(columns={'max': 'Max CLV', 'min': 'Min CLV', 'median': 'Median CLV'})  
print(clv_stats)

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [17]:
num_policies_state_month = df.groupby(['State','Policy'])['Customer'].count().reset_index()
pivot_table_num_policies = num_policies_state_month.pivot(index='State', columns='Policy', values='Customer')
print(pivot_table_num_policies)

Policy      Corporate L1  Corporate L2  Corporate L3  Personal L1  \
State                                                               
Arizona               62           119           200          293   
California           164           246           425          439   
Nevada                37            73           109          136   
Oregon                94           174           324          405   
Washington            40            46            89          125   

Policy      Personal L2  Personal L3  Special L1  Special L2  Special L3  
State                                                                     
Arizona             434          742          25          35          27  
California          905         1250          18          49          56  
Nevada              212          391           5          18          12  
Oregon              618         1157          21          63          53  
Washington          230          327           5          14      

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [ ]:


# Group by state and month to count the number of policies sold
policy_count_by_month = df.groupby(['State', 'Month'])['Customer'].count().reset_index(name='Policy Sold Count')

# Filter for the top 3 states with the highest number of policies sold
total_by_state = policy_count_by_month.groupby('State')['Policy Sold Count'].sum().reset_index()

top_3_states = total_by_state.sort_values(
    by='Policy Sold Count', 
    ascending=False).head(3)['State']

# Filter the data for only the top 3 states
filtered_data = policy_count_by_month[policy_count_by_month['State'].isin(top_3_states)]

# Pivot the data to display months as columns and states as rows
pivot_top3 = filtered_data.pivot(
    index='State',
    columns='Month',
    values='Policy Sold Count'
)
print(pivot_top3)

Month          1     2
State                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [30]:
response_rate = df.groupby('Sales Channel')['Response'].value_counts(normalize=True).unstack().fillna(0)
response_rate = (response_rate * 100).round(2)  # Convert to percentage
response_rate = response_rate.rename(columns={'Yes': 'Response Rate (%)'}).reset_index()
response_rate = response_rate.drop(columns=['No'])
print(response_rate)

Response Sales Channel  Response Rate (%)
0                Agent              19.07
1               Branch              11.38
2          Call Center              10.98
3                  Web              11.71


External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9